In [16]:
#%matplotlib notebook
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
import seaborn as sns
from sklearn.model_selection import train_test_split
%matplotlib inline
import math
from sklearn.model_selection import GridSearchCV, cross_val_score, learning_curve
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, Normalizer, RobustScaler
#import xgboost as xgb
#import lightgbm as lgb
import warnings

warnings.filterwarnings('ignore')

sns.set(style='white', context='notebook', palette='deep')

In [17]:
def normalization(data):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)
    return data

# Choosing data

In [18]:
df = pd.read_csv("new_dtrain.csv", delimiter=",")
zeros = df[df['latitude'] == 0]
df = df[df['latitude'] != 0]
subset = list(df.columns.values)
subset = subset.remove('price')
data = df.drop_duplicates(subset = subset)
#print(len(data[data.price > 1000000000]))
data = data[data.price < 1000000000]
data = data[data.price < 800000000]
data = data[data.price < 400000000]

data.replace(-1, 0, inplace=True)
data.replace(' С ', '0', inplace=True)

data = shuffle(data)
# data['sqr_meter'] = (data['price'] / data['all_space'])
# data['sqr_meter'] = pd.to_numeric(data['sqr_meter'], errors='coerce')

# def get_price_for_sqr_meter(dataframe):
#     dataframe = dataframe.copy()
#     dataframe[u'price_for_sqr_meter'] = dataframe.apply(calculate_price_for_sqr_meter, axis = 1)
#     return dataframe

# def calculate_price_for_sqr_meter(x):
#     return (x[u'price'] + 0.0) / x[u'all_space']

data.drop(32983, inplace=True)
# data = get_price_for_sqr_meter(data)

data = data.reset_index(drop=True)

In [19]:
# data = log_feature(data)
#choosing data
X = data[['room_number', 'house_type', 'district',
       'built_time', 'appartments_floor', 'all_space', 'state', 'bathroom',
       'balcony', 'balcony_glassed', 'door', 'phone', 'ceiling', 'safety',
       'at_the_hotel', 'internet', 'furniture', 'parking', 'building_floors', 'map_complex', 'floor',
          'trngl_first_point', 'trngl_second_point', 'trngl_third_point']]
# X_geo = data[['geocode_lat', 'geocode_long', 'trngl_first_point', 'trngl_second_point', 'trngl_third_point']]
#'latitude','longitude'
y = data['price']
# x_columns = X.columns
# x_index = X.index
# X = normalization(X)
# X = pd.DataFrame(X, index=x_index, columns=x_columns)
# X = pd.concat([X, X_geo], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)

# Just retrieving nearest (Cheating)

In [20]:
# def average_from_nearest(out):
#     from math import sin, cos, sqrt, atan2, radians
#     lat = X_train['latitude'].values
#     lon = X_train['longitude'].values
#     out_lat = out[latitude_index]
#     out_lon = out[longitude_index]
#     indx = X_train.index

#     answer = np.zeros(len(lat))

#     for i in range(len(lat)):
#         R = 6373.0
#         lat1 = radians(lat[i])
#         lon1 = radians(lon[i])
#         lat2 = radians(out_lat)
#         lon2 = radians(out_lon)

#         dlon = lon2 - lon1
#         dlat = lat2 - lat1

#         a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
#         c = 2 * atan2(sqrt(a), sqrt(1 - a))

#         distance = R * c

#         answer[i] = distance


#     newDf = pd.DataFrame()
#     newDf['distance'] = answer
#     newDf['indx'] = indx
#     newDf = newDf.sort_values(['distance'], ascending=True)
#     newDf = newDf[15:]
#     ind = newDf.indx
#     tofit = X_train.loc[ind.values]
#     avg = tofit['price_for_sqr_meter']
#     return np.average(avg)

In [21]:
# prices = np.zeros(len(X_test))
# X_test_m = X_test.as_matrix()
# latitude_index = X_test.columns.get_loc('latitude')
# longitude_index = X_test.columns.get_loc('longitude')

# for i in range(len(X_test)):
#      prices[i] = average_from_nearest(X_test_m[i])
# X_test['price_for_sqr_meter'] = prices

# Models 

In [22]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
# model = KNeighborsRegressor(n_neighbors = 5, n_jobs=-1, p=1)
# model.fit(X_train, y_train)

In [23]:
#gbr_model = GradientBoostingRegressor(n_estimators = 100, learning_rate=0.1, 
#                                  random_state=7, max_depth=10)
rfr_model = RandomForestRegressor(n_estimators = 200, random_state=7)
# logreg_model = LogisticRegression(C=0.5, n_jobs=-1)

In [24]:
#gbr_model.fit(X_train, y_train)
rfr_model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
           oob_score=False, random_state=7, verbose=0, warm_start=False)

# Metrics

In [25]:
# print(model.score(X_test, y_test))
#print(gbr_model.score(X_test, y_test))
print(rfr_model.score(X_test, y_test))

0.881045606255


In [26]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
print("RFR Mean absolute error: ", mean_absolute_error(rfr_model.predict(X_test), y_test))
#print("GBR Mean absolute error: ", mean_absolute_error(gbr_model.predict(X_test), y_test))
# print("KNN Regressor absolute error: ", mean_absolute_error(model.predict(X_test), y_test))

RFR Mean absolute error:  3234310.67637


# Overfitting

In [13]:
print("RFR Mean absolute error: ", mean_absolute_error(rfr_model.predict(X_train), y_train))
#print("GBR Mean absolute error: ", mean_absolute_error(gbr_model.predict(X_train), y_train))
# print("KNN Regressor absolute error: ", mean_absolute_error(model.predict(X_train), y_train))

RFR Mean absolute error:  97344.8578902


# Stacking

In [12]:
from sklearn.linear_model import LinearRegression, LogisticRegression
linear_model = LinearRegression()

In [13]:
def stack_out(data):
    stack = pd.DataFrame()
#     stack['knn'] = model.predict(data)
    stack['rfr'] = rfr_model.predict(data)
    stack['gbr'] = gbr_model.predict(data)
    return stack

In [14]:
linear_model.fit(stack_out(X_train), y_train)

NameError: name 'gbr_model' is not defined

In [ ]:
print("Stacking output(MAE): ", mean_absolute_error(linear_model.predict(stack_out(X_test)), y_test))

# Printing 

In [ ]:
g = sns.jointplot(x = data['district'], y = data['price'], kind="reg")

In [ ]:
g = sns.distplot(data['price'],color="grey")
g = g.legend(['Skewness : {:.2f}'.format(data['price'].skew())],loc='best')

In [ ]:
def RMSE(estimator,X_train, y_train, cv=5,n_jobs=4):
    cv_results = cross_val_score(estimator,X_train,y_train,cv=cv,scoring="neg_mean_squared_error",n_jobs=n_jobs)
    return (np.sqrt(-cv_results)).mean()

# Boosting Machines

In [ ]:
LightGB = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)


In [ ]:
X_train['room_number'] = pd.to_numeric(X_train['room_number'])
X_test['room_number'] = pd.to_numeric(X_test['room_number'])

In [ ]:
LightGB.fit(X_train, y_train)

In [ ]:
print("LightGBM MAE: ", mean_absolute_error(LightGB.predict(X_test), y_test))

In [ ]:
XGBoost = xgb.XGBRegressor()

In [ ]:
XGBoost.fit(X_train, y_train)

In [ ]:
print("LightGBM MAE: ", mean_absolute_error(XGBoost.predict(X_test), y_test))